In [1]:
using GittinsIndices
using Distributions
using Plots
using ProgressMeter

In [7]:
function get_gittins_action(gittins_priors, gamma)
    gittins_indices = [
        calculate_bernoulli_gittins(
            alpha = alpha, 
            beta = beta, 
            gamma = gamma,
        ) for (alpha, beta) in gittins_priors
    ]
    gittins_action = argmax(gittins_indices)
    return gittins_action
end

function get_thompson_action(thompson_priors)
    thompson_sampling_probs = [
        alpha / (alpha + beta) 
        for (alpha, beta) in thompson_priors
    ]
    thompson_sampling_probs ./= sum(thompson_sampling_probs)
    thompson_action = rand(Categorical(thompson_sampling_probs))
    return thompson_action
end

function get_ucb_action(ucb_arm_rewards, ucb_arm_counts, c = 1.0)
    total_count = sum(ucb_arm_counts)
    ucb_bounds = ucb_arm_rewards ./ ucb_arm_counts .+ c .* sqrt.(total_count ./ ucb_arm_counts)
    ucb_action = argmax(ucb_bounds)
    return ucb_action
end

get_ucb_action (generic function with 2 methods)

In [11]:
function gittins_vs(;num_arms, num_pulls)
    arms = [Bernoulli(rand(Float64)) for _ in 1:num_arms]
    best_arm_p = maximum([arm.p for arm in arms])
    optimal_rewards = collect(1:num_pulls) .* best_arm_p
    
    gittins_priors = [[1, 1] for _ in 1:num_arms]
    gittins_rewards = []
    
    thompson_priors = [[1, 1] for _ in 1:num_arms]
    thompson_rewards = []
    
    ucb_arm_rewards = [0.0 for _ in 1:num_arms]
    ucb_arm_counts = [1 for _ in 1:num_arms]
    ucb_rewards = []
    
    for pull in 1:num_pulls
        # gittins indices
        gamma = 1 - 1 / pull
        gittins_action = get_gittins_action(gittins_priors, gamma)
        gittins_reward = rand(arms[gittins_action])
        push!(gittins_rewards, gittins_reward)
        gittins_priors[gittins_action][gittins_reward ? 1 : 2] += 1        
        
        # thompson sampling
        thompson_action = get_thompson_action(thompson_priors)
        thompson_reward = rand(arms[thompson_action])
        push!(thompson_rewards, thompson_reward)
        thompson_priors[thompson_action][thompson_reward ? 1 : 2] += 1
            
        # UCB 
        ucb_action = get_ucb_action(ucb_arm_rewards, ucb_arm_counts)
        ucb_reward = rand(arms[ucb_action])
        push!(ucb_rewards, ucb_reward)
        ucb_arm_counts[ucb_action] += 1
        ucb_arm_rewards[ucb_action] += ucb_reward            
    end
            
    return (
        gittins_regret = optimal_rewards .- cumsum(gittins_rewards), 
        thompson_regret = optimal_rewards .- cumsum(thompson_rewards),
        ucb_regret = optimal_rewards .- cumsum(ucb_rewards),
    )
end

gittins_vs (generic function with 1 method)

In [ ]:
function graph_thompson_gittins()    
    num_arms = 5
    num_pulls = 100
    
    results = []
    @showprogress for _ in 1:100
        push!(
            results, 
            gittins_vs(
                num_arms = num_arms,
                num_pulls = num_pulls,
            )
        )
    end
    
    gittins_regret = mean([result.gittins_regret for result in results])
    thompson_regret = mean([result.thompson_regret for result in results])
    ucb_regret = mean([result.ucb_regret for result in results])
    
    plot(
        1:num_pulls, 
        [
            gittins_regret,
            thompson_regret,
            ucb_regret
        ], 
        title="Explore-Exploit Strategies for Multi-Armed Bandits", 
        label=["Gittins Indices" "Thompson Sampling" "UCB"],
        xlabel="Pulls",
        ylabel="Average Cumulative Regret",
    )

end

graph_thompson_gittins()

Progress:  95%|███████████████████████████████████████  |  ETA: 0:00:05